# Importing Data into Django

Let's say that you want to import a bunch of data into Django. To keep things simple, we'll say this is information about people -- names, addresses, etc.

In the simplest case you could simply assume that all of your source column names match your destination field names, and do something like this:

In [1]:
__ = Person.objects.all().delete()
__ = Case.objects.all().delete()

In [2]:
# Utility functions
def handle_form(data, form_class):
    form = form_class(data)
    if form.is_valid():
        return form.save()
    
    raise ValueError(form.errors.as_data())

In [3]:
from io import StringIO
import csv

from cases.forms import PersonForm

def create_person(data):
    person = handle_form(data, PersonForm)
    print(f"Created Person: {person}")
    return person

data = """name,phone,email
Thomas,123456789,thomas@thomas.com
Bill,758586998,foo@bar.com
"""

for row in csv.DictReader(StringIO(data)):
    print(f"Row: {row}")
    person = create_person(row)

Row: OrderedDict([('name', 'Thomas'), ('phone', '123456789'), ('email', 'thomas@thomas.com')])
Created Person: Thomas; 123456789; thomas@thomas.com
Row: OrderedDict([('name', 'Bill'), ('phone', '758586998'), ('email', 'foo@bar.com')])
Created Person: Bill; 758586998; foo@bar.com


Great, but what if they *don't* match? Easy, right? You can simply create a mapping of source field/column to destination field. For example:

In [4]:
def gen_reader(data, from_field_aliases):
    reader = csv.DictReader(StringIO(data))
    reader.fieldnames = [from_field_aliases[fieldname] for fieldname in reader.fieldnames]
    return reader

def process_csv(data, from_field_aliases):
    """Given a string representation of a CSV file, handle its contents"""
    reader = gen_reader(data, from_field_aliases)
    for row in reader:
        print(f"Row: {row}")
        person = create_person(row)

# Note that we are changing the headers here!
#         ↓ changed
data = """full_name,phone_number,email
Thomas,123456789,thomas@thomas.com
Bill,758586998,foo@bar.com
"""

from_field_aliases = {
    # from_field: alias
    "full_name": "name",
    "phone_number": "phone",
    "email": "email"
}


process_csv(data, from_field_aliases)

Row: OrderedDict([('name', 'Thomas'), ('phone', '123456789'), ('email', 'thomas@thomas.com')])
Created Person: Thomas; 123456789; thomas@thomas.com
Row: OrderedDict([('name', 'Bill'), ('phone', '758586998'), ('email', 'foo@bar.com')])
Created Person: Bill; 758586998; foo@bar.com


Good, this works just fine! But, what if we have data in many different files, often with different headers for the same columns?

In [5]:
# Data from the first CSV file
data1 = """full_name,phone_number,email
Thomas,123456789,thomas@thomas.com
"""

# Data from the second CSV file
data2 = """NAME,PHONE,EMAIL
Bill,758586998,foo@bar.com
"""

# We'll need to accommodate the fact that we have different headers representing the same thing.
# But, we can simply add more aliases to handle this
from_field_aliases = {
    "full_name": "name",
    "NAME": "name",
    "phone_number": "phone",
    "PHONE": "phone",
    "email": "email",
    "EMAIL": "email",
    "letter": "attachment",
    "LETTER": "attachment",
    "case_num": "case_num",
    "CASE": "case_num",
}

for data in [data1, data2]:
    process_csv(data, from_field_aliases)

Row: OrderedDict([('name', 'Thomas'), ('phone', '123456789'), ('email', 'thomas@thomas.com')])
Created Person: Thomas; 123456789; thomas@thomas.com
Row: OrderedDict([('name', 'Bill'), ('phone', '758586998'), ('email', 'foo@bar.com')])
Created Person: Bill; 758586998; foo@bar.com


But that's *very* verbose! We can actually "compress" this dict so that it can be represented more cleanly, although we'll need to invert the mapping and use `{from_field: [possible_from_fields]}` instead:

In [6]:
from pprint import pprint

compressed_from_field_aliases = {
    # from_field: (alias1, alias2, ...)
    "name": ("full_name", "NAME"),
    "phone": ("phone_number", "PHONE"),
    "email": ("email", "EMAIL"),
    "attachment": ("letter", "LETTER"),
    "case_num": ("case_num", "CASE"),
}

def invert_aliases(aliases_):
    """Given map of format {field, aliases, ...}, return one of format {alias: field, ...}"""
    return {alias: field for field, aliases in aliases_.items() for alias in aliases}

from_field_aliases = invert_aliases(compressed_from_field_aliases)
print("These are our expanded aliases:")
pprint(from_field_aliases)

print("And now to process the data, we do the same as before:")
for data in [data1, data2]:
    process_csv(data, from_field_aliases)

These are our expanded aliases:
{'CASE': 'case_num',
 'EMAIL': 'email',
 'LETTER': 'attachment',
 'NAME': 'name',
 'PHONE': 'phone',
 'case_num': 'case_num',
 'email': 'email',
 'full_name': 'name',
 'letter': 'attachment',
 'phone_number': 'phone'}
And now to process the data, we do the same as before:
Row: OrderedDict([('name', 'Thomas'), ('phone', '123456789'), ('email', 'thomas@thomas.com')])
Created Person: Thomas; 123456789; thomas@thomas.com
Row: OrderedDict([('name', 'Bill'), ('phone', '758586998'), ('email', 'foo@bar.com')])
Created Person: Bill; 758586998; foo@bar.com


But what about messy data *values*? For example, what if we encounter case numbers that cannot be converted to `float`? Well, we _could_ use Django's built-in form validation to handle this... but this quickly gets very messy once we get into more complicated cases (as we'll see soon).

So, instead, we'll introduce the concept of a "converter" function that handles this case.

In [7]:
data1 = """case_num
CASE#123123
"""
data2 = """CASE
CASE#456456
"""

from cases.forms import CaseForm

def create_case(data):
    case = handle_form(data, CaseForm)
    print(f"Created Case: {case}")
    return case

# Update our process_csv function to:
# * process Cases instead of People
# * convert any fields that need converting before sending them to the CaseForm
def process_csv(data, from_field_aliases):
    """Given a string representation of a CSV file, handle its contents"""
    reader = gen_reader(data, from_field_aliases)
    for row in reader:
        print(f"Row: {row}")
        # Convert all fields that have converters
        converted = {field: converters[field](value) for field, value in row.items() if field in converters}
        print("converted", converted)
        # Merge converted data with row data, overwriting any "old" data in the row
        case = create_case({**row, **converted})

    
compressed_from_field_aliases = {
    "case_num": ("case_num", "CASE"),
}

converters = {
    "case_num": lambda case_num: case_num.strip("CASE#"),
}

from_field_aliases = invert_aliases(compressed_from_field_aliases)
print("These are our expanded aliases:")
pprint(from_field_aliases)

print("And now to process the data, we do the same as before:")
for data in [data1, data2]:
    process_csv(data, from_field_aliases)

These are our expanded aliases:
{'CASE': 'case_num', 'case_num': 'case_num'}
And now to process the data, we do the same as before:
Row: OrderedDict([('case_num', 'CASE#123123')])
converted {'case_num': '123123'}
Created Case: #123123 (None)
Row: OrderedDict([('case_num', 'CASE#456456')])
converted {'case_num': '456456'}
Created Case: #456456 (None)


That works well enough, although we do have a bit of redundancy now -- two maps instead of a single map. Further, we still can't deal with more complicated mappings. If, for example, we wanted to combine latitude and longitude into a single field, location, how might we do that?

The primary change is that we need to move from the paradigm of "do something for each field in each row" to "do something for each mapping, for each row".

In [8]:
import itertools
from pprint import pprint, pformat

from django import forms
from cases.forms import StructureForm

def create_structure(data):
    structure = handle_form(data, StructureForm)
    print(f"Created Structure: {structure}")
    return structure

def process_csv(data, from_field_aliases):
    """Given a string representation of a CSV file, handle its contents"""
    reader = gen_reader(data, from_field_aliases)
    for row in reader:
        print(f"Row: {row}")
        # Convert all fields that have converters
        converted = {}
        for from_fields, converter in converters.items():
            fields_to_convert = {field: row[field] for field in from_fields}
            converted.update(converter(**fields_to_convert))
        print("converted", converted)
        data = {**row, **converted}
        print("data: ", data)
        # Merge converted data with row data, overwriting any "old" data in the row
        structure = create_structure(data)

data1 = """lat,long
38.1,72.8
"""

data2 = """LAT,LONG
38.2,78.5
"""

compressed_from_field_aliases = {
    # from_field: (alias1, alias2, ...)
    "latitude": ("lat", "LAT"),
    "longitude": ("long", "LONG"),
}


# Specify all custom converters here. Again, in cases where there is no conversion required,
# we don't need to specify anything at all.
converters = {
    # (from_field1, from_field2, ...): converter_function(from_field1, from_field2, ...)
    ("latitude", "longitude"): lambda latitude, longitude: {"location": f"{latitude},{longitude}"},
}

from_field_aliases = invert_aliases(compressed_from_field_aliases)
print("And now to process the data, we do the same as before:")
for data in [data1, data2]:
    process_csv(data, from_field_aliases)

And now to process the data, we do the same as before:
Row: OrderedDict([('latitude', '38.1'), ('longitude', '72.8')])
converted {'location': '38.1,72.8'}
data:  {'latitude': '38.1', 'longitude': '72.8', 'location': '38.1,72.8'}
Created Structure: 38.1,72.8
Row: OrderedDict([('latitude', '38.2'), ('longitude', '78.5')])
converted {'location': '38.2,78.5'}
data:  {'latitude': '38.2', 'longitude': '78.5', 'location': '38.2,78.5'}
Created Structure: 38.2,78.5


All of this is great, but what if want to process CSV files that contain data destined for multiple models? Our approach falls apart unless all fields in our models are unique in name. So, how can we fix this?

In [9]:
# Data from the first CSV file
data1 = """full_name,phone_number,email,case_num,lat,long
Thomas,123456789,thomas@thomas.com,CASE#123123,38.1,72.8
"""

# Data from the second CSV file
data2 = """NAME,PHONE,EMAIL,CASE,LAT,LONG
Bill,758586998,foo@bar.com,CASE#456456,38.2,78.5
"""

form_maps = {
    CaseForm: {
        "aliases": {
            "case_num": ("case_num", "CASE"),
        },
        "converters": {
            "case_num": lambda case_num: case_num.strip("CASE#"),
        }
    },
    PersonForm: {
        "aliases": {
            "name": ("full_name", "NAME"),
            "phone": ("phone_number", "PHONE"),
            "email": ("email", "EMAIL"),
        },
        "converters": {}
    },
    StructureForm: {
        "aliases": {
            "latitude": ("lat", "LAT"),
            "longitude": ("long", "LONG"),
        },
        "converters": {
            ("latitude", "longitude"): lambda latitude, longitude: {"location": f"{latitude},{longitude}"},
        }
    }
}

for form_class, info in form_maps.items():
    info["aliases"] = invert_aliases(info["aliases"])
    
pprint(form_maps)

def process_csv(data, from_field_aliases, form_maps):
    """Given a string representation of a CSV file, handle its contents"""
    for row in csv.DictReader(StringIO(data)):
        print(f"Row: {row}")
        for form_class, info in form_maps.items():
            # Convert all fields that have converters
            converted = {}
            data = {from_field: row[from_field] for from_field in info["aliases"].items()}
            print("data", data)
            for from_fields, converter in converters.items():
                fields_to_convert = {field: row[field] for field in from_fields}
                converted.update(converter(**fields_to_convert))
            data = {**data, **converted}
            handle_form(data, form_class)

# for data in [data1, data2]:
#     process_csv(data, from_field_aliases, form_maps)

{<class 'cases.forms.PersonForm'>: {'aliases': {'EMAIL': 'email',
                                                'NAME': 'name',
                                                'PHONE': 'phone',
                                                'email': 'email',
                                                'full_name': 'name',
                                                'phone_number': 'phone'},
                                    'converters': {}},
 <class 'cases.forms.CaseForm'>: {'aliases': {'CASE': 'case_num',
                                              'case_num': 'case_num'},
                                  'converters': {'case_num': <function <lambda> at 0x7ffb79df8d08>}},
 <class 'cases.forms.StructureForm'>: {'aliases': {'LAT': 'latitude',
                                                   'LONG': 'longitude',
                                                   'lat': 'latitude',
                                                   'long': 'longitude'},
                

Well, this is getting really complicated. Let's introduce a few abstractions to help us out!

First up is the concept of a `FieldMap`. This is a mapping of one or more "from" fields to one or more "to" fields. We've been representing these as dicts of `{ tuple : tuple }`, but wouldn't it be cleaner to bring all of this into a class?

Right, but what does it _do_?

In [10]:
from django_import_data import FieldMap
from django_import_data import OneToOneFieldMap, ManyToOneFieldMap

# Create a ManyToOneFieldMap for our location field
field_map = ManyToOneFieldMap(
    from_fields={
        "latitude": ("lat", "LAT"),
        "longitude": ("long", "LONG"),
    },
    to_field="location",
    converter=lambda latitude, longitude: (latitude, longitude)
)

# This will take process aliases, then call convert_location with latitude and longitude as arguments
field_map.render({"lat": "38.1", "longitude": "72.8"})


{'location': ('38.1', '72.8')}

Alright, that makes sense... but what we really need is a way to group these FieldMaps together sensibly. So, we'll introduce FormMap:

In [11]:
from django_import_data import FormMap
from django_import_data import OneToOneFieldMap, ManyToOneFieldMap

class CaseFormMap(FormMap):
    form_class = CaseForm
    field_maps = (
        OneToOneFieldMap(
            {"case_num": ("CASE",)}
        ),
    )
    
    def convert_case_num(self, case_num):
        return case_num.strip("CASE#")
    
class PersonFormMap(FormMap):
    form_class = PersonForm
    field_maps = (
        OneToOneFieldMap({"name": ("full_name", "NAME")}),
        OneToOneFieldMap({"phone": ("phone_number", "PHONE")}),
        OneToOneFieldMap({"email": ("EMAIL",)}),
    )
    

class StructureFormMap(FormMap):
    form_class = StructureForm
    field_maps = (
        ManyToOneFieldMap(
            from_fields={
                "latitude": ("lat", "LAT"),
                "longitude": ("long", "LONG"),
            },
            to_field="location",
            # This needs to be a string since the FieldMap is instantiated before the StructureFormMap,
            # and thus the function won't be bound correctly
            # We could also leave this out and rename the converter to convert_latitude_longitude
            converter="convert_location"
        ),
    )
    
    def convert_location(self, latitude, longitude):
        return {"location": (latitude, longitude)}

cfm = CaseFormMap()
case = cfm.save({"case_num": "456456"})
print(f"Created Case {case}")

pfm = PersonFormMap()
person = pfm.save({"name": "Bill", "phone": "758586998", "email": "foo@bar.com"})
print(f"Created Person {person}")
    
sfm = StructureFormMap()
structure = sfm.save({"lat": "38.2", "longitude": "78.5"})
print(f"Created Structure {structure}")

Created Case #456456 (None)
Created Person Bill; 758586998; foo@bar.com
Created Structure ('38.2', '78.5')
